<h1>Vorhersage, ob es bei einem Unfall Schwerverletzte/Tote gibt</h1>

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
crash_data = gpd.read_file('../01_Source/consolidatedDataSet.geojson') 

## 1. Vorbereitung des Datensatzes

### 1.1 Vorbereitung der Datenbasis für die konkrete Verarbeitung

In [2]:
crash_data = crash_data.drop(["ULAND","UREGBEZ","UJAHR","LINREFX","LINREFY","METER","EW2010","GFZ","HERKUNFT","NAME","KNAME","PLR_NAME",
                              "VBUS","VBUS_PERS","GESLAERM_PERS","GEMEINDE","BEZIRK","ANWERT","VERFAHRENSART","STICHTAG","BEITRAGSZUSTAND","LUMNUM","ELEM_NR","VRICHT_TXT","ZEIT_T","TAG_T"
                             ,"DURCH_T","DANN_T","DAT_T","LAT","LON","formatted_address","address_components","street_number","route","postal_code","sublocality","BRW","UART","UWOCHENTAG","ULICHTVERH","UTYP1","STRZUSTAND"], axis=1)
crash_data = crash_data.drop(["OBJECTID","STR_NAME"], axis=1)

In [3]:
crash_data.head()

,UKREIS,UGEMEINDE,UMONAT,USTUNDE,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,DTV,GESLAERM,NUTZUNG,WERT_VES,TARGET,UART_Value,UWOCHENTAG_Value,ULICHTVERH_Value,UTYP1_Value,STRZUSTAND_Value,ANTEIL_PKW,ANTEIL_LKW,ANTEIL_LIEFERWAGEN,ANTEIL_LINIENBUSSE,ANTEIL_REISEBUSSE,ANTEIL_KRAD,UKREIS_AVG_TARGET,UGEMEINDE_AVG_TARGET,CRASH_BY_DTV,FATAL_CRASH_BY_DTV,Indicator,geometry
0,10,10,3,11,0,1,0,0,0,0,19575.767816,mittel,W - Wohngebiet,30.0,0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Donnerstag,Tageslicht,Einbiegen / Kreuzen-Unfall,trocken,82.408776,2.762958,2.762958,2.762958,0.400124,2.494525,21.266234,21.266234,5.108,0.000,0.029628,"MULTIPOLYGON (((814997.310 5829184.012, 814996..."
1,10,10,1,18,0,1,0,0,0,0,6108.000000,mittel,W - Wohngebiet,30.0,0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Montag,Dunkelheit,Einbiegen / Kreuzen-Unfall,trocken,81.777996,3.601834,10.216110,2.619515,0.261952,1.522593,21.266234,21.266234,153.846,51.282,0.148718,"MULTIPOLYGON (((814885.204 5830057.780, 814884..."
2,10,10,6,7,1,1,0,0,0,0,19575.767816,niedrig - sehr niedrig,W - Wohngebiet,30.0,1,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Mittwoch,Tageslicht,Einbiegen / Kreuzen-Unfall,trocken,82.408776,2.762958,2.762958,2.762958,0.400124,2.494525,21.266234,21.266234,10.217,5.108,0.029628,"MULTIPOLYGON (((814281.955 5828040.742, 814281..."
3,10,10,6,13,0,1,0,0,1,0,19575.767816,mittel,W - Wohngebiet,30.0,0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Mittwoch,Tageslicht,Einbiegen / Kreuzen-Unfall,trocken,82.408776,2.762958,2.762958,2.762958,0.400124,2.494525,21.266234,21.266234,5.108,0.000,0.029628,"MULTIPOLYGON (((814215.008 5830068.156, 814214..."
4,10,10,5,20,0,1,0,0,0,0,3852.000000,mittel,W - Wohngebiet,30.0,0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Samstag,Dämmerung,Einbiegen / Kreuzen-Unfall,trocken,81.178609,2.777778,10.202492,3.894081,0.415369,1.531672,21.266234,21.266234,153.846,51.282,0.148718,"MULTIPOLYGON (((814765.428 5829893.128, 814764..."


In [4]:


#crash_data = crash_data.drop(["OBJECTID","STR_NAME"], axis=1)


crash_data_copy = crash_data.copy()
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UART_Value"], prefix='UART_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UWOCHENTAG_Value"], prefix='UWOCHENTAG_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UTYP1_Value"], prefix='UTYP1_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["ULICHTVERH_Value"], prefix='ULICHTVERH_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["STRZUSTAND_Value"], prefix='STRZUSTAND_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["NUTZUNG"], prefix='NUTZUNG_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["GESLAERM"], prefix='GESLAERM_')

#corr_matrix = crash_data_copy.corr()
#corr_matrix["TARGET"]
crash_data_copy = crash_data_copy.drop(["geometry"], axis=1)

### 1.2 Train-Test Split

In [5]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

X = crash_data_copy.drop(["TARGET"] , axis=1)
Y = crash_data_copy["TARGET"]
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=12, ratio = 1.0)
x_train_res, y_train_res = sm.fit_sample(X_train, y_train)

### 2.1 Trainieren und Testen eines RandomForestClassifiers

In [6]:
crash_data_copy["TARGET"].value_counts()

0    11433
1     2216
Name: TARGET, dtype: int64

In [7]:
(11433-2216)/11433

0.806175107145981

In [50]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier()

param_grid = {
                 'n_estimators': [17, 50, 90, 95, 100, 115],
                 'max_depth': [1, 2,3, 5, 7, 9, 11, 13],
                 'class_weight':["balanced"],
                 'min_samples_leaf': [2, 3, 4, 5, 7, 9]
             }

param_grid2 = {
                 'n_estimators': [85, 90, 95],
                 'max_depth': [3, 4, 5],
                 'min_samples_leaf': [7] #5
             }

from sklearn.model_selection import GridSearchCV

grid_clf = GridSearchCV(clf, param_grid, cv=10, verbose=10, n_jobs=5, scoring='f1')
grid_clf.fit(X_train, y_train)

#Train the model using the training sets y_pred=clf.predict(X_test)
#clf.fit(X_train,y_train)

y_pred=grid_clf.predict(X_test)

Fitting 10 folds for each of 288 candidates, totalling 2880 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    1.6s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    1.7s
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    1.9s
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    2.3s
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    2.8s
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    3.2s
[Parallel(n_jobs=5)]: Done  51 tasks      | elapsed:    4.1s
[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed:    4.6s
[Parallel(n_jobs=5)]: Done  75 tasks      | elapsed:    4.9s
[Parallel(n_jobs=5)]: Done  88 tasks      | elapsed:    5.6s
[Parallel(n_jobs=5)]: Done 103 tasks      | elapsed:    6.5s
[Parallel(n_jobs=5)]: Done 118 tasks      | elapsed:    7.6s
[Parallel(n_jobs=5)]: Done 135 tasks      | elapsed:    8.0s
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed:    9.2s
[Parallel(n_jobs=5)]: Done 171 tasks      | elapsed:   10.6s
[Parallel(

In [51]:
y_pred=grid_clf.predict(X_test)

In [52]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7347985347985349


In [53]:
#grid_clf.best_params_
#{'max_depth': 1, 'n_estimators': 5}
from sklearn.metrics import f1_score
f1_score = f1_score(y_test, y_pred)

print('F1 score: {0:0.2f}'.format(
      f1_score))

F1 score: 0.42


In [54]:
grid_clf.best_params_

{'class_weight': 'balanced',
 'max_depth': 11,
 'min_samples_leaf': 9,
 'n_estimators': 115}

In [55]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)

(1745, 540, 184, 261)

In [57]:
clf=RandomForestClassifier(random_state=42,class_weight='balanced',max_depth=11, min_samples_leaf=9, n_estimators=115)

clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [58]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7395604395604396


In [59]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)

(1766, 519, 192, 253)

In [60]:
#Precision
tp/(tp+fp)

0.327720207253886

In [61]:
#Recall
tp/(tp+fn)

0.5685393258426966

### 2.2. Wichtige Features für Klassifikationsentscheidung

In [62]:
feature_imp = pd.Series(clf.feature_importances_, index= X_train.columns).sort_values(ascending=False)

In [63]:
feature_imp

FATAL_CRASH_BY_DTV                                                               0.295460
CRASH_BY_DTV                                                                     0.054806
UART_Value__Zusammenstoß mit vorausfahrendem / wartendem Fahrzeug                0.046143
ANTEIL_KRAD                                                                      0.036820
ANTEIL_LIEFERWAGEN                                                               0.034391
UTYP1_Value__Überschreitenunfall                                                 0.031288
ANTEIL_REISEBUSSE                                                                0.031048
ANTEIL_PKW                                                                       0.030446
ANTEIL_LKW                                                                       0.030401
Indicator                                                                        0.028716
DTV                                                                              0.027695
ANTEIL_LIN